# OpenAI Assistant with Autogen and Postgres

This example hows how you can use the OpenAI Assistant agents with autogen, to solve database related tasks (Postgres DB).

In [1]:
import os
from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv())

from modules import llm
from modules import rand
from turbo4 import Turbo4
from modules import embeddings
from modules.instruments import PostgresAgentInstruments

import autogen
from autogen.agentchat.contrib.gpt_assistant_agent import GPTAssistantAgent


DB_URL = os.environ.get("POSTGRES_CONNECTION_URL")
POSTGRES_TABLE_DEFINITIONS_CAP_REF = "TABLE_DEFINITIONS"

In [2]:
# ! pip install --upgrade pyautogen
# https://github.com/microsoft/autogen/releases/tag/0.2.0b5

In [3]:
raw_prompt = "Get the mean age of those who have had covid."
prompt = f"Fulfill this database query: {raw_prompt}. "

assistant_name = "test"
session_id = rand.generate_session_id(assistant_name + raw_prompt)
assistant_id = Turbo4().get_or_create_assistant(assistant_name).assistant_id

get_or_create_assistant(test, gpt-4-1106-preview)


In [4]:
# Define the SQL tool schema
run_sql_tool_config = {
    "type": "function",
    "function": {
        "name": "run_sql",
        "description": "Run a SQL query against the postgres database",
        "parameters": {
            "type": "object",
            "properties": {
                "sql": {
                    "type": "string",
                    "description": "The SQL query to run",
                }
            },
            "required": ["sql"],
        },
    },
}

# Configuration for GPTAssistantAgent
config_list = autogen.config_list_from_dotenv(
    dotenv_file_path='../../.env',
    model_api_key_map={
        "gpt-4-1106-preview": "OPENAI_API_KEY",
    },
    filter_dict={
        "model": {
            "gpt-4-1106-preview",
        }
    }
)

# Initialize User Proxy Agent
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    system_message= "You are a helpful AI Assistant who runs code to get answers. You use the provided python tools to run SQL, you pass the code as a string and get back the result.",
    is_termination_msg=lambda msg: "TERMINATE" in msg["content"],
    code_execution_config={
        "work_dir": "./autogen_results",
        "use_docker": False
    },
    human_input_mode="NEVER",
)

In [5]:
SQL_ASSISTANT_SYSTEM_MESSAGE = """You are a helpful AI assistant.
Solve tasks using your coding and language skills.
In the following cases, suggest SQL code (in a SQL coding block) or shell script (in a sh coding block) for the user to execute.
    1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
    2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
    3. Code must consist the SQl being passed to the provided run_sql tool
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
You must tell the user that they must use the tool to run the string of SQL. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple SQL code blocks in one response. Do not ask users to copy and paste the result. Check the execution result returned by the user.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible. 
Reply "TERMINATE" in the end when everything is done.
"""

In [6]:
with PostgresAgentInstruments(DB_URL, session_id) as (agent_instruments, db):
    
    llm_config = {
        "config_list": config_list,
        "assistant_id": assistant_id,
        "tools": [run_sql_tool_config],
        "model": "gpt-4-1106-preview"
    }

    # Initialize the assistant
    sql_assistant = GPTAssistantAgent(
        name="SQL_Assistant",
        instructions=SQL_ASSISTANT_SYSTEM_MESSAGE,
        llm_config=llm_config
    )

    # Register the run_sql function
    sql_assistant.register_function(
        function_map={
            "run_sql": agent_instruments.run_sql
        }
    )

    # Get table definitions from Postgres
    database_embedder = embeddings.DatabaseEmbedder(db)
    table_definitions = database_embedder.get_similar_table_defs_for_prompt(
        raw_prompt
    )

    # Forumalte prompt
    prompt = llm.add_cap_ref(
        prompt,
        f"Use these {POSTGRES_TABLE_DEFINITIONS_CAP_REF} to satisfy the database query.",
        POSTGRES_TABLE_DEFINITIONS_CAP_REF,
        table_definitions,
    )

    # Initiate autogen chat
    user_proxy.initiate_chat(sql_assistant, message=prompt)

user_proxy (to SQL_Assistant):

Fulfill this database query: Get the mean age of those who have had covid..  Use these TABLE_DEFINITIONS to satisfy the database query.

TABLE_DEFINITIONS

CREATE TABLE patients (
id integer,
patient_id character varying(255),
state character varying(2),
fips integer,
diagnosed_covid boolean,
diagnoses_date date,
current_age integer,
age_at_diagnosis double precision
);

CREATE TABLE patient_medical_history (
id integer,
patient_id character varying(255),
has_diabetes boolean,
has_heart_disease boolean,
has_lung_disease boolean,
smoker_status character varying(10),
bmi double precision,
last_checkup_date date
);

--------------------------------------------------------------------------------
SQL_Assistant (to user_proxy):

To calculate the mean age of those who have had COVID-19, we need to select the `current_age` from the `patients` table where `diagnosed_covid` is `true` and then calculate the average of these ages. Here is the SQL code that you need

KeyboardInterrupt: 